# A la carte analysis at jupyter.nsls2 with tiled

### Andi Barbour, Soft X-ray Scattering and Spectroscopy, CSX, NSLS-II

### NSLS-II and CFN Users' Meeting 2020
Workshop 6

Notebook #2

In [ ]:
from matplotlib import cm, pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import datetime
from itertools import cycle

# Explore data produced by bluesky using tiled
## 2D scans from pre-assembled plans


[current bluesky documentation](https://blueskyproject.io/bluesky/)

[current tiled documentation](https://blueskyproject.io/tiled/)

In [ ]:
from databroker.queries import TimeRange, RawMongo
from tiled.client import from_uri
c = from_uri("https://tiled-demo.blueskyproject.io/api")
csx = c["csx"]["raw"]

### for your NSLS-II experiment using databroker v1
```python
from databroker import Broker
from databroker import list_profiles
list_profiles()


db = Broker.named('csx')
```

### for your NSLS-II experiment using tiled
```python
from tiled.client import from_profile
c = from_profile("csx")
```



In [ ]:
dict_slt3_x_pinhole = {-6: '50 um', 
                       11: '10 um'}

dict_esdiag_xray_measure = {-15:'flux', 
                            -59:'drain current'} #Cu L3 resonance

In [ ]:
scan = 151860
run = csx[scan]
start_time = run.start["time"]
print(scan, start_time, "-->", datetime.datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
data_xr = run["primary"]["data"].read(['nanop_bz_user_setpoint', 'nanop_bx_user_setpoint', 'dif_beam_stats2_total'])
print(run) #151876

## Explore the data and try to plot something
- scanned motors are
    - `'nanop_bz_user_setpoint'`
    - `'nanop_bx_user_setpoint'`
- counting "point" detector is ROI 2 from `dif_beam` AreaDetector
    - `'dif_beam_stats2_total'`
    
*Here is some room to try things out*
- Can you plot something?
- Do you now what kind of scan or the plan pattern?

### possible solutions

In [ ]:
data_xr

In [ ]:
print(f'{list(data_xr)}\n')

print(f'{list(data_xr.coords)}\n')

In [ ]:
f = plt.figure()
for mtr_pos in list(data_xr)[0:2]:
    plt.scatter(data_xr.coords["time"], data_xr[mtr_pos], label=data_xr[mtr_pos].name)
plt.legend()
plt.ylabel('motor position'); plt.xlabel('time')
f.savefig('motor_positions_standard_plot.png')

In [ ]:
run.metadata["start"]

### We saved a figure

- `"time"` is like a sequential index

- We didn't need to access the start document to see `snake_axes=True`


[mesh_grid() with snaking argument](https://blueskyproject.io/bluesky/tutorial.html#scan-multiple-motors-in-a-grid)

### Can we exploint this `"time"` index?

In [ ]:
print(f'{data_xr.dims["time"]}')
print(f'{np.arange(data_xr.dims["time"])}')

In [ ]:
f2 = plt.figure()
for mtr_pos in list(data_xr)[0:2]:
    data_xr.plot.scatter("time", mtr_pos, label=mtr_pos)
plt.legend()
plt.ylabel('motor position')
f2.savefig('motor_positions_xarray_plot.png')

In [ ]:
data_xr.dif_beam_stats2_total.plot()

In [ ]:
data_xr

### We know w will see some central peak if we can create a 3d-plot
-**What matplotlib pyplot functions do you reach for?**

[matplotlib gallery](https://matplotlib.org/stable/gallery/index.html)

-**Will any pyplot function work with the data as is?**

### possible solutions

In [ ]:
data_xr.plot.scatter("nanop_bx_user_setpoint", "nanop_bz_user_setpoint", c = data_xr.dif_beam_stats2_total, marker="s", s=100)

In [ ]:
data_xr = run["primary"]["data"].read(['nanop_bz_user_setpoint', 'nanop_bx_user_setpoint', 'dif_beam_stats2_total'])
df_data = data_xr.to_dataframe()
df_data = df_data.set_index([ "nanop_bz_user_setpoint", "nanop_bx_user_setpoint"])
ds_img = xr.Dataset.from_dataframe(df_data)

print(f'{data_xr.dims}')
print(f'{ds_img.dims}')

ds_img

In [ ]:
#__________________
x = ds_img.nanop_bx_user_setpoint
y = ds_img.nanop_bz_user_setpoint
z = ds_img.dif_beam_stats2_total
plt.figure()
plt.pcolormesh(x, y, z, )

#______________
plt.figure()
ds_img.dif_beam_stats2_total.plot.pcolormesh()

In [ ]:
#______________
plt.figure()
ds_img.dif_beam_stats2_total.plot.imshow()

##  Now that we no what and how to plot:
- what data did are coworker collect while we were:
    - sleeping
    - eating
- data we collected that we were too tired to write down or our notes were confusing.
    - We know we sorted the problem, but we don't quite remember the details
    - We are unsure if we picked the best configuration

### Try to extract these beamtime data only knowing:
    - the first scan is `151860`
    - the scans occurred until midnight
    

### possible solutions

In [ ]:
start_time = run.start["time"]
print(scan, start_time, "-->", datetime.datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S'), "\n")



In [ ]:
runs = csx.search(TimeRange(since = "2022-02-10 20:35:47", until = "2022-02-11 00:00:00"))
#runs = csx.search(TimeRange(since = "2022-02-10 20:35:47", until = "2022-02-11 00:00:00")).search(RawMongo(start={"plan_name": 'rel_grid_scan',}))
#runs = csx.search(TimeRange(since = "2022-02-10 20:35:47", until = "2022-02-11 00:00:00")).search(RawMongo(start={"plan_name": 'rel_grid_scan', "detectors":'dif_beam'}))
#runs = csx.search(TimeRange(since = "2022-02-10 20:35:47", until = "2022-02-11 00:00:00")).search(RawMongo(start={"plan_name": 'rel_grid_scan', "detectors":'sclr'}))
runs

In [ ]:
for k, v in [my_dict.items() for my_dict in [dict_slt3_x_pinhole, dict_esdiag_xray_measure]]:
    print(k, v)


In [ ]:
#from IPython.display import display -- don't seem to need at jupyter.nsls2 anymore
name_order = ["tardis_delta","tardis_theta", "pgm_energy_setpoint", "slt3_x_user_setpoint", "es_diag1_y_user_setpoint"]
#name_order = [ "pgm_energy_setpoint", "slt3_x_user_setpoint", "es_diag1_y_user_setpoint"]
for run in runs.values():
    df = run["baseline"]["data"].read(name_order).to_dataframe()
    print(run.start["scan_id"], run.start["detectors"], run.start["plan_name"], run.start["motors"], run.start["num_points"])# "--"*10)
    print("--"*50)
    if df is not None:
        display(df[name_order].mean()) # FOR USER TO TRY - what happens when you remove .mean()
    
    print(f'{run.stop["exit_status"]:>60} {run.metadata["summary"]["duration"]/60:.2f} minutes')# "--"*10)
    print("\n")

### Maybe you prefer plots or a different table

In [ ]:
runs = csx.search(TimeRange(since = "2022-02-10 20:35:47", until = "2022-02-11 00:00:00"))
#runs = csx.search(TimeRange(since = "2022-02-10 20:35:47", until = "2022-02-11 00:00:00")).search(RawMongo(start={"plan_name": 'rel_grid_scan',}))
#runs = csx.search(TimeRange(since = "2022-02-10 20:35:47", until = "2022-02-11 00:00:00")).search(RawMongo(start={"plan_name": 'rel_grid_scan', "detectors":'dif_beam'}))
#runs = csx.search(TimeRange(since = "2022-02-10 20:35:47", until = "2022-02-11 00:00:00")).search(RawMongo(start={"plan_name": 'rel_grid_scan', "detectors":'sclr'}))

### possible solution for plot

In [ ]:
colors = cycle(cm.get_cmap('viridis')(np.linspace(0, 1, len(runs))))

name_order = [ "pgm_energy_setpoint", "slt3_x_user_setpoint", "es_diag1_y_user_setpoint"]
fig, axes = plt.subplots(1,len(name_order), figsize=(10,5))
for run in runs.values():
    df = run["baseline"]["data"].read(name_order).to_dataframe()
    df = df.mean()
    color = next(colors)
    
    for i, name in enumerate(name_order):
        axes[i].plot(run.start["scan_id"], df[name],"o", ms=10, color=color )#label=f'{run.start["detectors"]} {run.start["plan_name"]} {run.start["motors"]} {run.start["num_points"]})
        
    for ax, name in zip(axes, name_order):
        ax.set(title=name, xlabel='scan_id' )
plt.suptitle("Summary of Beamline Configuration Parameters")

### Let's look at some targeted scans

In [ ]:
scans_final = [151863 , 151868 ]

In [ ]:
scans_final.extend([ 151865,151867])

runs_final = csx[scans_final]
runs_final

In [ ]:
def summarize_runs(runs, name_order):
    for run in runs:
        df = run["baseline"]["data"].read(name_order).to_dataframe()
        print(run.start["scan_id"], run.start["detectors"], run.start["plan_name"], run.start["motors"], run.start["num_points"])# "--"*10)
        print("--"*50)
        if df is not None:
            display(df[name_order].mean())

        print(f'{run.stop["exit_status"]:>60} {run.metadata["summary"]["duration"]/60:.2f} minutes')# "--"*10)
        print("\n")

In [ ]:
summarize_runs(runs_final, name_order = ["pgm_energy_setpoint", "slt3_x", "es_diag1_y_user_setpoint" ])

### Try plotting data

### more matplotlib style

In [ ]:
name_order_bl = ["pgm_energy_setpoint", "slt3_x_user_setpoint"]
name_order = ['nanop_bz_user_setpoint', 'nanop_bx_user_setpoint', 'dif_beam_stats2_total', 'sclr_ch2']

fig, axes = plt.subplots(2,2)

for run in csx[scans_final]:
    data_bl = run.baseline.data.read(name_order_bl)
    data = run.primary.data.read()#(name_order)
    detectors = run.metadata["start"]["detectors"]
    i, j = 0, 0
    if data_bl["pgm_energy_setpoint"].mean() > 931:
        i = 1
    if data_bl["slt3_x_user_setpoint"].mean() > 0:
        j = 1
    
    data_df = data.to_dataframe().set_index([ "nanop_bz_user_setpoint", "nanop_bx_user_setpoint"])
    data_img = xr.Dataset.from_dataframe(data_df)
    if detectors[0] == 'dif_beam':
        signal = data_img.dif_beam_stats2_total
    elif detectors[0] == 'sclr':
        signal = data_img.sclr_ch2 /  data_img.sclr_ch1

    ax = axes[j,i]
    ax.pcolormesh(data_img.nanop_bx_user_setpoint, data_img.nanop_bz_user_setpoint,signal)

### using xarray plotting features

In [ ]:
# for scan in scans_final:
#     run = csx[scan]
for run in csx[scans_final]:
    data_bl = run.baseline.data.read(name_order_bl)
    data = run.primary.data.read()#(name_order)
    detectors = run.metadata["start"]["detectors"]
    
    data_df = data.to_dataframe().set_index([ "nanop_bz_user_setpoint", "nanop_bx_user_setpoint"])
    data_img = xr.Dataset.from_dataframe(data_df)
    if detectors[0] == 'dif_beam':
        signal = data_img.dif_beam_stats2_total
    elif detectors[0] == 'sclr':
        signal = data_img.sclr_ch2 #/  data_img.sclr_ch1
    
    plt.figure()
    signal.plot.imshow()
    df = data_bl.to_dataframe()
    energy = df["pgm_energy_setpoint"].mean()
    slt3 = round(df["slt3_x_user_setpoint"].mean())
    
    plt.title(f'Energy = {energy}eV - {dict_slt3_x_pinhole[slt3]} pinhole')

### Stay Tuned
[more on slicing with tiled](https://blueskyproject.io/tiled/tutorials/slicing.html)